# Exploring Australian Radar Data

_In this tutorial you'll learn how to access the Australia weather radar dataset and generate basic plots_

### Topics

1. Overview of Australian Radar Dataset
2. Accessing level 1 data
3. Basic plotting


![radar logo](https://s3-ap-southeast-2.amazonaws.com/public-web-data/ausradar-logo.png)

Joshua Soderholm 2018

---

## 1. Overview

The Australian radar dataset is hosted on NCI and is a collabration between Monash, Bureau of Meteorology and NCI. This dataset was built by combining multiple Bureau of Meteorology radar archives of volumetric radar data and repairing corrupt volumes.
Multiple products are under development that relate to the level of processing. This includes:

- Level 0: Source rapic archives


- **Level 1: Unified archive that uses the ODIM model of the HDF5 data format. Daily summary images and statistics are also generated for this level.**


- Level 1b PPI: Processed level 1 dataset
     1. Corrections (Dealiasing, clutter removal)
     1. Reflectivity calibration
     1. Volumetric retrievals (azimuthal shear, rain rate)
     
     
- Level 1b Grids: Gridded version of Level 1b PPI for which two grid resolutions provided: 150 km radius @ 1000 m horiztonal res. All grids use a 500 m vertical resolution to an altitude of 20 km.
    1. corrected + calibrated reflectivity
    1. rain rate and azimuthal shear
    
  
- Level 2: Daily Grids
    1. 2.5 km Steiner classification
    1. Surface rainfall
    1. 0.5 km reflectivity surface (corrected and calibrated)
    1. Maximum Estimated Size of Hail (MESH)
    1. 0-6 km Azimuthal shear

_Only level 1 data is provided in the open access portal. This notebook will explore this data._

### Level 1 Description

Level 1 data is accessible on NCI via the THREDDS service, which allows for access via http requests.

**Path**

The root http path of the level 1 data is
http://dapds00.nci.org.au/thredds/fileServer/rq0/
Subfolder levels are orgnaised as:
1. radar id (two digits from 01 to 79) [reference spreadsheet](https://docs.google.com/spreadsheets/d/1XQimogwIN78FecQhqSRfgzF4zzMxIEP8guaR-lPY7eQ/edit?usp=sharing)
1. year (four digits)
1. Output type (string: 'vol' for radar volumes, 'list' for statistics and 'img' for daily images)

**Outputs**
1. radar volumes: Arranged into daily zip files with the naming convention ID_yyyymmdd.pvol.zip
    - e.g., http://dapds00.nci.org.au/thredds/fileServer/rq0/50/2014/vol/50_20141127.pvol.zip
1. statistics: comma delimated text files where each row represents a radar volume. First row header provides information on column fields.
1. daily images: PNG files containing the accumulated maximum relfectivity for each day

**Radar Volumes**

A daily tarball will typically contain a 240 or 144 radar volumes depending on volume scan time (6 or 10 minutes), with the naming convection ID_yyyymmdd_HHMMSS.pvol.h5

Downloading, extracting, loading and plotting of radar volumes can all be done automatically using Python!

Let's get started...

---

## 2. Data Access


**Tasks**
1. Specifiy a level 1 file to download
2. Download the tar file
3. Extract the tar file
4. List the extracted files

In [1]:
%matplotlib inline

import os #used for system commands
import tempfile #used to create temporary folders to store data
import zipfile #used to extract tar files
import urllib #used to download data via http
from datetime import datetime #used to manipulate time :)
from glob import glob #used for manipulating pathnames

import numpy as np 
from matplotlib import pyplot as plt

import pyart

from matplotlib import animation
from IPython.display import HTML
import cartopy.crs as ccrs # A toolkit for map projections


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/home/meso/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Building the request url

#Specific the radar and date we want to download
radar_id     = 50 #this is the Marburg radar near Brisbane.
date         = '2014/11/27' #in yyyy/mm/dd
base_url     = 'http://dapds00.nci.org.au/thredds/fileServer/rq0' #base url for NCI dataset

#parse inputs
radar_id_str = str(radar_id).zfill(2) #convert radar id to a string and fill with a leading 0 if only one digit
date_dt      = datetime.strptime(date,'%Y/%m/%d')

#build request filename url
tar_fn       = radar_id_str + '_' + date_dt.strftime('%Y%m%d') + '.pvol.zip'
request_url  = '/'.join([base_url, 'odim_pvol', radar_id_str, date_dt.strftime('%Y'), 'vol', tar_fn])
print('my request is ',request_url)

my request is  http://dapds00.nci.org.au/thredds/fileServer/rq0/odim_pvol/50/2014/vol/50_20141127.pvol.zip


In [3]:
#downloading and extracting the data

#download the zip file
if not os.path.isfile(tar_fn):
    urllib.request.urlretrieve(request_url, tar_fn)

#extract the zip file to a temporary directory
temp_dir = tempfile.mkdtemp()
zip_fh = zipfile.ZipFile(tar_fn)
zip_fh.extractall(path = temp_dir)
zip_fh.close()

#list all the volumes extracted from the zip file
file_list = sorted(glob(temp_dir + '/*'))
print('\n'.join(file_list)) #print with newline between each list item

/tmp/tmpyd0nddku/50_20141127_000152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_001152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_002152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_003152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_004152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_005152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_010152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_011152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_012152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_013152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_014152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_015152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_020152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_021152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_022152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_023152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_024152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_025152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_030152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_031152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_032152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_033152.pvol.h5
/tmp/tmpyd0nddku/50_20141127_034

## 3. Basic Plotting
Tasks
1. Specifiying a time range to subset list of volumes
1. Loading selected volumes using py-art
1. Basic plotting and exporting to an animation

In [4]:
#specify a time range subset and find the files we want to plot

#specify start and end times for plotting a subset of radar volumes
start_str = '2014/11/27 05:00' #time in UTC
end_str   = '2014/11/27 08:00' #time in UTC
#specify radar tilt and field to plot
tilt      = 1 #second tilt!
field     = 'DBZH' #reflectivity

#first convert the start/end time strings into datetime numbers
start_dt  = datetime.strptime(start_str, '%Y/%m/%d %H:%M')
end_dt    = datetime.strptime(end_str, '%Y/%m/%d %H:%M')

#now let's read the datetime numbers of all the volumes for comparision
file_dt_list = []
for i, fname in enumerate(file_list):
    file_dt_list.append(datetime.strptime(os.path.basename(fname)[3:18],'%Y%m%d_%H%M%S'))
    
#find the index of volumes within our start and end times
file_dt_array = np.array(file_dt_list)
index_array = np.where(np.logical_and(file_dt_array >= start_dt, file_dt_array<=end_dt))[0]

#time to plot
print(file_dt_array[index_array])


[datetime.datetime(2014, 11, 27, 5, 1, 52)
 datetime.datetime(2014, 11, 27, 5, 11, 52)
 datetime.datetime(2014, 11, 27, 5, 21, 52)
 datetime.datetime(2014, 11, 27, 5, 31, 52)
 datetime.datetime(2014, 11, 27, 5, 41, 52)
 datetime.datetime(2014, 11, 27, 5, 51, 52)
 datetime.datetime(2014, 11, 27, 6, 1, 52)
 datetime.datetime(2014, 11, 27, 6, 11, 52)
 datetime.datetime(2014, 11, 27, 6, 21, 52)
 datetime.datetime(2014, 11, 27, 6, 31, 51)
 datetime.datetime(2014, 11, 27, 6, 41, 51)
 datetime.datetime(2014, 11, 27, 6, 51, 52)
 datetime.datetime(2014, 11, 27, 7, 1, 52)
 datetime.datetime(2014, 11, 27, 7, 11, 52)
 datetime.datetime(2014, 11, 27, 7, 21, 52)
 datetime.datetime(2014, 11, 27, 7, 31, 52)
 datetime.datetime(2014, 11, 27, 7, 41, 52)
 datetime.datetime(2014, 11, 27, 7, 51, 52)]


In [5]:
#build list of radar objects to plot
radar_list = []
for index in index_array:
    #get file name of index
    file_name = file_list[index]
    #open volume using pyart
    my_radar = pyart.aux_io.read_odim_h5(file_name, file_field_names=True)
    #clean up field metadata
    my_radar.fields['DBZH']['standard_name'] = 'Reflectivity'
    my_radar.fields['DBZH']['units'] = 'dBZ'
    my_radar.fields['DBZH']['long_name'] = 'Radar Reflectivity Factor'
    #append to radar list for animation later
    radar_list += [my_radar]

#determine plot domains
radar_lat = my_radar.latitude['data'][0]
radar_lon = my_radar.longitude['data'][0]
min_lat   = radar_lat - 1.0
max_lat   = radar_lat + 1.0
min_lon   = radar_lon - 1.5
max_lon   = radar_lon + 1.5



In [6]:
#generate animation of reflectivity

# Set up the GIS projection
projection = ccrs.Mercator(
                central_longitude=radar_lon,
                min_latitude=min_lat, max_latitude=max_lat)

# Function which generates a plot for each sweep
def animate_reflectivity(nframe):
    plt.clf()
    #load diplay class for radar using cartopy
    display = pyart.graph.RadarMapDisplayCartopy(radar_list[nframe])
    display.plot_ppi_map('DBZH', tilt,
                            projection=projection, colorbar_flag=False,
                            min_lon=min_lon, max_lon=max_lon, min_lat=min_lat, max_lat=max_lat,
                            vmin=-8, vmax=64, cmap=pyart.graph.cm_colorblind.HomeyerRainbow,
                            resolution='10m')
    #here is our pretty colorbar code
    lb = display._get_colorbar_label('DBZH')
    cb = plt.colorbar(display.plots[0], aspect=30, pad=0.07, 
                      orientation='horizontal')
    cb.ax.tick_params(labelsize=20)
    cb.set_label(lb, fontsize=20)

    #Now we add lat lon lines
    gl = display.ax.gridlines(draw_labels=True,
                              linewidth=2, color='gray', alpha=0.5,
                              linestyle='--')
    gl.xlabel_style = {'size': 20}
    gl.ylabel_style = {'size': 20}

    gl.xlabels_top = False
    gl.ylabels_right = False
    
#Generate animation
fig = plt.figure(figsize=(16, 12))
anim = animation.FuncAnimation(fig, animate_reflectivity, frames=len(radar_list))
#Save animation to gif
anim_name = 'dbzh_animation_' + datetime.now().strftime('%H%M%S') +'.gif'
anim.save(anim_name,
          writer='imagemagick', fps=2)
plt.close()
#show gif in notebook
HTML('<img src="' + anim_name + '">')